In the upper right corner, select from the dropdown menu 'Change Runtime Type', and select a GPU as available - this will allow you to run your neural network training on accelerated hardware and run everything faster.

In [1]:
import numpy as np
import torch
from torch import nn
import random
import matplotlib.pyplot as plt
import torch.optim as optim
from collections import deque
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


This indicates that a GPU has been detected and can be used - the device is saved to 'device' so that we can direct data and models to access memory on that device.

In [2]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

train_X = torch.Tensor( trainset.data/255.0 - 0.5 )
train_X = train_X.permute( 0, 3, 1, 2 )

train_X = train_X.to( device ) # This line is different from the previous CIFAR code - it transfers the tensor to the GPU memory

test_X = torch.Tensor( testset.data/255.0 - 0.5 )
test_X = test_X.permute( 0, 3, 1, 2 )

test_X = test_X.to( device ) # Again, transfering the tensor to GPU memory.

train_Y = torch.Tensor( np.asarray( trainset.targets ) ).long()
train_Y = train_Y.to( device )
test_Y = torch.Tensor( np.asarray( testset.targets ) ).long()
test_Y = test_Y.to( device )

# All the data needs to be loaded into the GPU, as that is where the model processing will occur.

def get_batch(x, y, batch_size):
  n = x.shape[0]

  batch_indices = random.sample( [ i for i in range(n) ], k = batch_size )

  x_batch = x[ batch_indices ]
  y_batch = y[ batch_indices ]

  return x_batch, y_batch

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class CIFARModel(nn.Module):
  def __init__(self):
    super(CIFARModel, self).__init__()

    self.conv_layer_1 = nn.Conv2d(in_channels = 3, out_channels = 5, kernel_size = 3, stride = 1, bias=True)
    self.conv_layer_2 = nn.Conv2d(in_channels = 5, out_channels = 10, kernel_size = 3, stride = 1, bias=True)
    self.conv_layer_3 = nn.Conv2d(in_channels = 10, out_channels = 15, kernel_size = 3, stride = 1, bias=True)

    self.linear_layer = torch.nn.Linear( in_features = 15*26*26, out_features = 10, bias=True )
    # Note that the output of the last convolutional layer will be 15x26x16 - why?
    # So we want to input 15*26*26 values into the last layer, and get 10 output values out (for the class probabilities)

  def forward(self, input_tensor):
    output = self.conv_layer_1( input_tensor )
    output = nn.Sigmoid()( output )
    output = self.conv_layer_2( output )
    output = nn.Sigmoid()( output )
    output = self.conv_layer_3( output )
    output = nn.Sigmoid()( output )

    # At this point, the block of node values from the convolutional layer is flattened
    # So that it can be passed into a standard linear layer
    output = nn.Flatten()( output )
    output = self.linear_layer( output )
    return output

In [4]:
def confusion_matrix( model, x, y ):
  identification_counts = np.zeros( shape = (10,10), dtype = np.int32 )

  logits = model( x )
  predicted_classes = torch.argmax( logits, dim = 1 )

  n = x.shape[0]

  for i in range(n):
    actual_class = int( y[i].item() )
    predicted_class = predicted_classes[i].item()
    identification_counts[actual_class, predicted_class] += 1

  return identification_counts

In [5]:
cifar_model = CIFARModel()

cifar_model.to( device ) # The only change is that we also send the model to the GPU

print( cifar_model )
confusion_matrix( cifar_model, test_X, test_Y )

CIFARModel(
  (conv_layer_1): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer_2): Conv2d(5, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer_3): Conv2d(10, 15, kernel_size=(3, 3), stride=(1, 1))
  (linear_layer): Linear(in_features=10140, out_features=10, bias=True)
)


array([[   0,    0,    0,    0, 1000,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 1000,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 1000,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 1000,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 1000,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 1000,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 1000,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 1000,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 1000,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0, 1000,    0,    0,    0,    0,    0]])

At this point, everything else runs as before - just faster. I also increased the bach size, which I might could have done previously. Nevertheless - faster.

In [6]:
cnn_optimizer = optim.Adam(cifar_model.parameters(), lr = 0.01 )
loss_function = torch.nn.CrossEntropyLoss()
print("Initial Test Loss:", loss_function( cifar_model( test_X ), test_Y ).item() )

Initial Test Loss: 2.3501369953155518


In [9]:
batch_size = 16

for epochs in range(100):
  total_loss = 0
  for batch in range( train_X.shape[0] // batch_size ):
    x_batch, y_batch = get_batch(train_X, train_Y, batch_size)

    cnn_optimizer.zero_grad()
    logits = cifar_model( x_batch )
    loss = loss_function( logits, y_batch )

    loss.backward()
    cnn_optimizer.step()

    total_loss += loss.item()

  print( "Average Total Loss over Batches:", total_loss / ( train_X.shape[0] // batch_size ) )
  print( confusion_matrix( cifar_model, test_X, test_Y ) )

Average Total Loss over Batches: 0.15405796080672068
[[498  66  59  29  33  19  15  21 190  70]
 [ 63 581  13  19  13   9  22  20  88 172]
 [ 93  19 325 127 167 104  65  41  34  25]
 [ 34  34 108 324  95 159  76  79  34  57]
 [ 54  20 129 105 381  70  70 130  22  19]
 [ 17  18  97 204  77 406  46  80  26  29]
 [ 22  29  88 124  91  68 495  30  20  33]
 [ 35  25  68  79 107  96  22 481  11  76]
 [146  89  32  21  17  13   9  17 580  76]
 [ 52 187  14  26  17  16  13  34  77 564]]
Average Total Loss over Batches: 0.17345946182014882
[[480  54  71  38  45  33  16  35 149  79]
 [ 57 546  20  28  19  21  30  30  70 179]
 [ 70  12 320 129 157 131  97  44  19  21]
 [ 23  19  97 327  88 207 100  82  21  36]
 [ 36  14 127 116 369  82  94 131  15  16]
 [ 11  10  83 189  66 457  53  95  17  19]
 [ 11  14  72 111  79  84 569  34   6  20]
 [ 23  12  70  74 108 116  31 519   3  44]
 [160  79  33  26  26  26  20  24 528  78]
 [ 45 166  19  38  27  28  20  53  57 547]]
Average Total Loss over Batches:

Similar results - but much faster.